In [1]:
# import importants
import pandas as pd
import pickle
import re
import time
import random
import requests
import bs4
from bs4 import BeautifulSoup as bs
import json

In [2]:
# initalize lists

links = []
dict_list = []

In [3]:
# 12 pages
# 96 computers to a page

all_pages = ['https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=2',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=3',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=4',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=5',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=6',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=7',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=8',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=9',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=10',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=11',
            'https://www.newegg.com/p/pl?N=100019096%204814%20600014655%20600014654%20600014652%20600014656&Order=3&PageSize=96&page=12']

In [5]:
# function to scrape from  individual link

def scrape_pipeline(url):
    time.sleep(2 + 2*random.random())                    # sleep timer
    response = requests.get(url)                         # boil soup
    page = response.text
    soup = bs(page)
    rawColumns = soup.find_all('th')                     # extract content from soup
    rawElements = soup.find_all('td')
    key_cols = []                                        # initalize key and val lists
    val_els = [] 
    key_cols = [i.text for i in rawColumns]              # fill lists
    val_els = [i.text for i in rawElements]   
    res = {} 
    for key in key_cols:                                 # form dictionary from key and val lists
        for value in val_els:
            res[key] = value
            val_els.remove(value)
            break         
    price = soup.find_all(text = re.compile('"price":')) # get price & append to res dictionary
    try:
        new_dict = json.loads(str(price[0]))
        res['Price'] = new_dict['offers']['price']
    except:
        pass
    dict_list.append(res)                                # append current dict to global variable dict_list  

In [6]:
# get inner links to each computer
# add to a list

for page in all_pages:
    time.sleep(2 + 2*random.random())
    response = requests.get(page)
    page_html = response.text
    soup = bs(page_html)
    links_one = [link['href'] for link in soup.find_all('a', class_="item-title")]
    links.append(links_one)
    
len(links)

32

In [7]:
with open('pickled_links_list', 'wb') as f:
    pickle.dump(links,f)

In [8]:
with open('pickled_links_list', 'rb') as f:
    links_list = pickle.load(f)

len(links_list)

32

In [9]:
flat_list = [item for sublist in links_list for item in sublist]

len(flat_list) # expecting

0

In [10]:
for link in flat_list:
    scrape_pipeline(link)

In [11]:
len(dict_list)

0

In [12]:
df = pd.DataFrame(dict_list)

In [13]:
with open('pickled_df', 'wb') as f:
    pickle.dump(df,f)

In [14]:
# End of Webscraping
# Start Step_2 - Data Cleaning